# Example: Spark + DSE

In BGDS, we advocate for the convination of the following tools as the best solution to address Big data's problems.

- Apache Spark
- Apache Cassandra (DSE)
- Apache Solr (DSE)

This notebook is an example of how to use these tools together to analyze the data crawled by the `Bovespa` example crawler included in the DaVinci distribution ().

In [1]:
import IPython
IPython.auto_scroll_threshold = 9999

from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"
from IPython.display import display, HTML

%load_ext autoreload
%autoreload 2
%load_ext autotime

In [2]:
from pyspark.sql.functions import *
from pyspark.sql import Window

time: 428 ms


In [101]:
from financials import compute_zscore, group_by_period
from spark import fillna
from db import sync_table

time: 557 ms


In [5]:
from spark import init_spark_context, load_and_get_table_df

sc, sql_context = init_spark_context("Factor Analysis Job - CapEx Vol 6Q")

time: 6.6 s


## Using the Quarterly calendar

We will work with a quarterly trading calendar. Computing the returns and factors using this periodicity.

In [44]:
# The dates of the analysis
start_analysis_date = "2008-03-31"
end_analysis_date = "2015-12-31"

# Ontain the list of companies in our universe
calendar = load_and_get_table_df(sql_context, "tfm_uoc_analysis", "quarterly_trading_dates"). \
    filter(col("calendar") == "BMF").select(col("first_date"), col("last_date")). \
    filter((col("last_date") >= start_analysis_date) & (col("last_date") <= end_analysis_date))
calendar.cache()
calendar.show()

DataFrame[first_date: date, last_date: date]

+----------+----------+
|first_date| last_date|
+----------+----------+
|2008-01-02|2008-03-31|
|2008-04-01|2008-06-30|
|2008-07-01|2008-09-30|
|2008-10-01|2008-12-30|
|2009-01-02|2009-03-31|
|2009-04-01|2009-06-30|
|2009-07-01|2009-09-30|
|2009-10-01|2009-12-30|
|2010-01-04|2010-03-31|
|2010-04-01|2010-06-30|
|2010-07-01|2010-09-30|
|2010-10-01|2010-12-30|
|2011-01-03|2011-03-31|
|2011-04-01|2011-06-30|
|2011-07-01|2011-09-30|
|2011-10-03|2011-12-29|
|2012-01-02|2012-03-30|
|2012-04-02|2012-06-29|
|2012-07-02|2012-09-28|
|2012-10-01|2012-12-28|
+----------+----------+
only showing top 20 rows

time: 297 ms


In [7]:
# Ontain the list of companies in our universe
universe = load_and_get_table_df(sql_context, "tfm_uoc_analysis", "portfolio_universe")
universe.cache()

DataFrame[ccvm: string, ticker: string, EBIT: decimal(38,18), EarningsYield: double, ROC: decimal(38,18), astodate: date, cash: decimal(38,18), current_assets: decimal(38,18), current_liabilities: decimal(38,18), current_non_cash_assets: decimal(38,18), dividend: double, excess_cash: decimal(38,18), fixed_liabilities: decimal(38,18), liquidity120days: double, marketcap: double, price_share: float, quality_index: double, short_term_investments: decimal(38,18), solr_query: string, stock: decimal(38,18), total_cash: decimal(38,18), total_debt: decimal(38,18), total_shares: decimal(38,18)]

time: 7.13 s


In [72]:
# Accessing to the fundamental data of the companies
# Attaching a data frame to the accounting notes available in the database
fundamentals = load_and_get_table_df(sql_context, "tfm_uoc", "bovespa_account")

factor_fdata = fundamentals.filter(
    col("number").isin(["6.02", "1"])). \
    join(universe.select("ccvm", "ticker"), "ccvm", how="right"). \
    orderBy(asc("period"), asc("ticker"))
              
factor_fdata.cache()

# Tagging account numbers with names (they will become the column names)
factor_fdata = factor_fdata.withColumn(
    "factor_name", when(factor_fdata.number == "1", "total_assets").otherwise("cap_ex_reported"))

factor_fdata = factor_fdata\
    .select(col("ticker").alias("asset"),
            col("period").alias("astodate"),
            col("factor_name"), 
            col("amount").alias("amount")). \
    orderBy(asc("astodate"), asc("asset"))

factor_fdata = factor_fdata.groupby(col("asset"), col("astodate"))\
    .pivot("factor_name").sum("amount"). \
    filter((col("astodate") >= start_analysis_date) & (col("astodate") <= end_analysis_date)). \
    orderBy("asset", "astodate", ascending=[True, True])

factor_fdata.cache()
factor_fdata.show(20)

DataFrame[ccvm: string, period: date, version: string, number: string, financial_info_type: string, amount: decimal(38,18), balance_type: string, comments: string, created_at: timestamp, deleted_reason: string, is_deleted: boolean, name: string, sector: int, solr_query: string, updated_at: timestamp, ticker: string]

DataFrame[asset: string, astodate: date, cap_ex_reported: decimal(38,18), total_assets: decimal(38,18)]

+-----+----------+--------------------+--------------------+
|asset|  astodate|     cap_ex_reported|        total_assets|
+-----+----------+--------------------+--------------------+
|ABEV3|2012-12-31|               0E-18|1346301.000000000...|
|ABEV3|2013-03-31|-4123.00000000000...|1340374.000000000...|
|ABEV3|2013-06-30|-1231619.00000000...|85378102.00000000...|
|ABEV3|2013-09-30|-2709273.00000000...|98307922.00000000...|
|ABEV3|2013-12-31|-7618992.00000000...|225113706.0000000...|
|ABEV3|2014-03-31|2109545.000000000...|127286233.0000000...|
|ABEV3|2014-06-30|468742.0000000000...|127249408.0000000...|
|ABEV3|2014-09-30|-1686164.00000000...|133835159.0000000...|
|ABEV3|2014-12-31|-3645869.00000000...|149557315.0000000...|
|ABEV3|2015-03-31|-3256538.00000000...|160390910.0000000...|
|ABEV3|2015-06-30|-4541695.00000000...|157996588.0000000...|
|ABEV3|2015-09-30|-6624377.00000000...|187943833.0000000...|
|ABEV3|2015-12-31|-11095752.0000000...|184898978.0000000...|
|BBAS3|2010-12-31|-35531

We have the required fundamental data about all the companies in our restricted universe.

Now it's time to compute the signal of the alpha factor. 

In [73]:
# We scale the capex using the total assets of each company
# if we do so we will be able to compare the level of capex
# between companies
factor_signal = factor_fdata.withColumn(
    "cap_ex_reported_scaled", 
    when(factor_fdata.cap_ex_reported > 0, 0).
    otherwise(abs(factor_fdata.cap_ex_reported)) / factor_fdata.total_assets)

factor_signal = factor_signal.withColumn(
    "capex_vol_6q", stddev(col("cap_ex_reported_scaled"))
    .over(Window.partitionBy("asset").rowsBetween(-5, 0)) )

factor_signal.cache()
factor_signal.show(20)

DataFrame[asset: string, astodate: date, cap_ex_reported: decimal(38,18), total_assets: decimal(38,18), cap_ex_reported_scaled: decimal(38,6), capex_vol_6q: double]

+-----+----------+--------------------+--------------------+----------------------+--------------------+
|asset|  astodate|     cap_ex_reported|        total_assets|cap_ex_reported_scaled|        capex_vol_6q|
+-----+----------+--------------------+--------------------+----------------------+--------------------+
|ENBR3|2010-12-31|-3273510.00000000...|88056400.00000000...|              0.037175|                 NaN|
|ENBR3|2011-03-31|-239510.000000000...|35144842.00000000...|              0.006815| 0.02146776187682358|
|ENBR3|2011-06-30|6904.000000000000...|17202536.00000000...|              0.000000|0.019791220486198757|
|ENBR3|2011-09-30|-241180.000000000...|17563372.00000000...|              0.013732|  0.0161661720164876|
|ENBR3|2011-12-31|-1095708.00000000...|37093768.00000000...|              0.029539| 0.01554548785660971|
|ENBR3|2012-03-31|-226170.000000000...|18581060.00000000...|              0.012172|0.014070412437688765|
|ENBR3|2012-06-30|-323550.000000000...|18284884.0000000

We need to rank the alpha signal to have a more robust version of the signal. Which is able to stay out of outliers, handle noise in the data and also keep us from making potential excessive trades.

In [74]:
factor_signal_ranked = factor_signal. \
    orderBy("astodate", "asset", ascending=[True, True]). \
    withColumn("capex_vol_6q_ranked", rank().over(
        Window.partitionBy("astodate").orderBy(asc("capex_vol_6q"))))

factor_signal_ranked.cache()
factor_signal_ranked.select("asset", "astodate", "capex_vol_6q", "capex_vol_6q_ranked").show(20)

DataFrame[asset: string, astodate: date, cap_ex_reported: decimal(38,18), total_assets: decimal(38,18), cap_ex_reported_scaled: decimal(38,6), capex_vol_6q: double, capex_vol_6q_ranked: int]

+-----+----------+--------------------+-------------------+
|asset|  astodate|        capex_vol_6q|capex_vol_6q_ranked|
+-----+----------+--------------------+-------------------+
|BBSE3|2015-12-31|                 0.0|                  1|
|BRAP4|2015-12-31|                 0.0|                  1|
|MRVE3|2015-12-31|                 0.0|                  1|
|BRPR3|2015-12-31|4.025328143973689E-4|                  4|
|BRML3|2015-12-31|0.001491207117293...|                  5|
|BBAS3|2015-12-31|0.005716945653056359|                  6|
|RENT3|2015-12-31|0.005739632546310493|                  7|
|ODPV3|2015-12-31|0.006009125194235847|                  8|
|MRFG3|2015-12-31|0.007184968856346328|                  9|
|ITUB4|2015-12-31|0.008450682946760378|                 10|
|PCAR4|2015-12-31|0.009919104766392312|                 11|
|CYRE3|2015-12-31|0.013284883961354974|                 12|
|IGTA3|2015-12-31|0.014055394144835167|                 13|
|BRFS3|2015-12-31| 0.01595268485240024| 

We standardize the alpha signal to help us compare different alpha peformances and be able to combine them.

In [75]:
factor_signal_ranked_std = compute_zscore(factor_signal_ranked, "astodate", "capex_vol_6q_ranked")
factor_signal_ranked_std.cache()
factor_signal_ranked_std.select(
    "asset", "astodate", "capex_vol_6q", 
    "capex_vol_6q_ranked", "capex_vol_6q_ranked_zscored").show(20)

DataFrame[asset: string, astodate: date, cap_ex_reported: decimal(38,18), total_assets: decimal(38,18), cap_ex_reported_scaled: decimal(38,6), capex_vol_6q: double, capex_vol_6q_ranked: int, capex_vol_6q_ranked_zscored: double]

+-----+----------+--------------------+-------------------+---------------------------+
|asset|  astodate|        capex_vol_6q|capex_vol_6q_ranked|capex_vol_6q_ranked_zscored|
+-----+----------+--------------------+-------------------+---------------------------+
|BBSE3|2015-12-31|                 0.0|                  1|         -1.642393787847823|
|BRAP4|2015-12-31|                 0.0|                  1|         -1.642393787847823|
|MRVE3|2015-12-31|                 0.0|                  1|         -1.642393787847823|
|BRPR3|2015-12-31|4.025328143973689E-4|                  4|        -1.3225767419383079|
|BRML3|2015-12-31|0.001491207117293...|                  5|        -1.2159710599684694|
|BBAS3|2015-12-31|0.005716945653056359|                  6|         -1.109365377998631|
|RENT3|2015-12-31|0.005739632546310493|                  7|        -1.0027596960287926|
|ODPV3|2015-12-31|0.006009125194235847|                  8|        -0.8961540140589541|
|MRFG3|2015-12-31|0.007184968856

We now generate the dataframe with the alpha signal alone. The dataframe will be compose by dates as indexes and the name of the stocks as columns. The cells will contains the computed alpha signal.

In [76]:
capex_vol_6q_signal = factor_signal_ranked_std.withColumn(
    "capex_vol_6q", factor_signal_ranked_std.capex_vol_6q_ranked_zscored). \
    select(col("astodate").alias("date"), "asset", "capex_vol_6q"). \
    orderBy("asset", "date", ascending=[True, True])

capex_vol_6q_signal = capex_vol_6q_signal.groupBy(col("date")). \
    pivot("asset").sum("capex_vol_6q"). \
    orderBy(["date"], ascending=[True])

capex_vol_6q_signal.cache()

DataFrame[date: date, ABEV3: double, BBAS3: double, BBSE3: double, BEEF3: double, BRAP4: double, BRFS3: double, BRML3: double, BRPR3: double, CCRO3: double, CIEL3: double, CSNA3: double, CYRE3: double, ECOR3: double, ENBR3: double, IGTA3: double, ITUB4: double, JBSS3: double, MRFG3: double, MRVE3: double, MULT3: double, NATU3: double, ODPV3: double, OIBR4: double, PCAR4: double, QUAL3: double, RENT3: double, TOTS3: double, UGPA3: double, USIM5: double, VALE3: double, VIVT4: double, VLID3: double]

time: 12.8 s


We need to make sure all the cells have values. We fast forward fill the missing values, and fill with 0 (no signal) the initial missings of each ticker. 

In [97]:
for asset_name in capex_vol_6q_signal.columns[1:]:
    capex_vol_6q_signal = fillna(capex_vol_6q_signal, 
           partition_keys=None, 
           order_field="date", 
           field_to_fill=asset_name,
           fill_function=last,
           use_same_field_as_output=True)

    capex_vol_6q_signal = capex_vol_6q_signal.withColumn(
            asset_name,
            when((isnan(col(asset_name)) | (isnull(col(asset_name)))), 0.0).
            otherwise(col(asset_name)))

capex_vol_6q_signal.cache()

DataFrame[date: date, ABEV3: double, BBAS3: double, BBSE3: double, BEEF3: double, BRAP4: double, BRFS3: double, BRML3: double, BRPR3: double, CCRO3: double, CIEL3: double, CSNA3: double, CYRE3: double, ECOR3: double, ENBR3: double, IGTA3: double, ITUB4: double, JBSS3: double, MRFG3: double, MRVE3: double, MULT3: double, NATU3: double, ODPV3: double, OIBR4: double, PCAR4: double, QUAL3: double, RENT3: double, TOTS3: double, UGPA3: double, USIM5: double, VALE3: double, VIVT4: double, VLID3: double]

time: 7.25 s


We are ready, we can convert the spark dataframe into a pandas dataframe

In [98]:
capex_vol_6q_signal_pdf = capex_vol_6q_signal.toPandas()
capex_vol_6q_signal_pdf = capex_vol_6q_signal_pdf. \
    sort_values(['date'], ascending=[True])
capex_vol_6q_signal_pdf = capex_vol_6q_signal_pdf.set_index("date")

capex_vol_6q_signal_pdf

,ABEV3,BBAS3,BBSE3,BEEF3,BRAP4,BRFS3,BRML3,BRPR3,CCRO3,CIEL3,...,OIBR4,PCAR4,QUAL3,RENT3,TOTS3,UGPA3,USIM5,VALE3,VIVT4,VLID3
date,,,,,,,,,,,,,,,,,,,,,
2010-12-31,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
2011-03-31,0.000000,-1.083473,0.000000,0.505621,0.650084,-0.505621,-1.372399,1.516862,1.227936,0.000000,...,-0.794547,0.000000,1.661325,-1.516862,-0.361158,-0.650084,0.000000,0.794547,-0.216695,0.000000
2011-06-30,0.000000,-1.372399,0.000000,-0.072232,0.794547,-0.650084,0.505621,1.516862,1.227936,0.000000,...,-1.083473,0.000000,1.661325,-1.227936,-0.505621,-0.939010,0.000000,0.650084,0.361158,0.000000
2011-09-30,0.000000,-1.372399,0.000000,-0.216695,0.650084,-0.794547,1.083473,1.516862,1.227936,0.000000,...,-1.083473,0.000000,1.661325,-1.227936,-0.650084,-0.505621,0.000000,0.505621,0.361158,0.000000
2011-12-31,0.000000,-1.213110,0.000000,-0.288836,0.519904,-0.404370,1.097575,1.559712,1.328644,-1.097575,...,-0.635438,-1.328644,1.675247,-0.982041,-0.519904,0.288836,0.750973,0.635438,0.404370,-0.057767
2012-03-31,0.000000,-1.372399,0.000000,0.216695,0.361158,-0.794547,1.227936,1.516862,0.939010,-1.097575,...,-0.939010,-1.328644,1.661325,-1.227936,-0.650084,-0.072232,0.750973,0.505621,0.072232,-0.057767
2012-06-30,0.000000,-1.083473,0.000000,0.650084,-1.661325,-0.072232,1.661325,0.361158,-0.216695,-1.097575,...,0.216695,-1.328644,1.227936,-0.939010,0.505621,0.794547,0.750973,0.939010,-0.794547,-0.057767
2012-09-30,0.000000,-1.227936,0.000000,0.650084,-1.661325,-0.216695,1.661325,0.794547,0.216695,-1.097575,...,-0.072232,-1.328644,1.227936,-0.794547,0.361158,0.505621,0.750973,0.939010,-0.939010,-0.057767
2012-12-31,1.582558,-1.357936,1.582558,0.057857,-1.684658,-0.486679,1.146929,0.275671,0.493486,-0.051050,...,-0.268865,-0.595586,1.038022,-0.922308,-0.159957,0.166764,0.711300,0.384578,-1.466844,1.473650


time: 3.72 s


In [107]:
capex_vol_6q_signal_pdf.to_csv("capex_vol_6q_factor.csv")

time: 12.7 ms


# Persist the results

Now it's time to save the factor into a table. To do this, the system should had create a 
keyspace for us in the database. The keyspace should have the same TOKEN than the one used
to connect through the API.

Each user will have it's own space in the system to run the analyses.

```sql
CREATE KEYSPACE IF NOT EXISTS <USER_ID>_analysis WITH replication = {'class': 'SimpleStrategy', 'replication_factor' : 1};
```

with full access to it:

```sql
-- Create the user using the token and
CREATE ROLE user_<USER_ID> WITH PASSWORD = '<USER_TOKEN>' AND LOGIN = true AND SUPERUSER = false;

-- Grant all permission for the user keyspace
GRANT ALL PERMISSIONS IN KEYSPACE <USER_ID>_analysis to user_<USER_ID>;
```

PRO features:
- Allow replication of data (ReplicationFactor = 3)
    ```sql
    CREATE KEYSPACE IF NOT EXISTS <TOKEN>_analysis WITH replication = {'class': 'SimpleStrategy', 'replication_factor' : 3};
    ```
- Network replication (multiple DataCenters)
    ```sql
    CREATE KEYSPACE IF NOT EXISTS <TOKEN>_analysis WITH replication = {'class': 'NetworkTopologyStrategy', 'DC1' : 2, 'DC2': 3, 'DC3': 2};
    ```

In [103]:
sync_table(capex_vol_6q_signal, "tfm_uoc_dse", "tfm_uoc_analysis", "capex_vol_6q_factor", "date")

/opt/conda/lib/python3.7/site-packages/dse/cqlengine/management.py:540: UserWarning: CQLENG_ALLOW_SCHEMA_MANAGEMENT environment variable is not set. Future versions of this package will require this variable to enable management functions.
  warnings.warn(msg)


Closing connections
time: 2.1 s


In [105]:
capex_vol_6q_signal.write\
    .format("org.apache.spark.sql.cassandra")\
    .options(table="capex_vol_6q_factor", keyspace="tfm_uoc_analysis")\
    .option("confirm.truncate","true")\
    .mode("overwrite")\
    .partitionBy("astodate")\
    .save()

time: 2.13 s


In [108]:
sc.stop()

time: 7.7 s
